Import all Required library

In [1]:
import time
import pandas as pd
import transformers
import numpy as np
from transformers import AutoTokenizer
from utils import read_data_as_sentence,map_labels_in_dataframe,tokenize_and_align_labels

## Preprocessing data

Read data files and save them as dataframe.\
\
Use `read_data_as_sentence` function from `utils.py` to read files as Dataframe.\

`read_data_as_sentence` need two input:
1. path of conllu file.
2. path for save datarame

In [2]:
train_data = read_data_as_sentence('data/en_ewt-up-train.conllu', 'data/en_ewt-up-train.preprocessed.csv')
dev_data = read_data_as_sentence('data/en_ewt-up-dev.conllu', 'data/en_ewt-up-dev.preprocessed.csv')
test_data = read_data_as_sentence('data/en_ewt-up-test.conllu', 'data/en_ewt-up-test.preprocessed.csv')

Show 20 input form and their gold list in test data

In [41]:
for i in range(20):
    print(f"{test_data.input_form[i]}\t:\t{test_data.argument[i]}")

['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?', '[SEP]', 'morph']	:	['_', '_', 'ARG1', '_', '_', 'ARG2', '_', None, '_']
['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand']	:	['_', '_', 'ARG0', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'ARG1', '_', '_', '_', '_', '_', '_', 'ARG4', '_', None, '_']
['(', 'And', ',', 'by', 'the', 'way', ',', 'is', 'anybody', 'else', 'just', 'a', 'little', 'nostalgic', 'for', 'the', 'days', 'when', 'that', 'was', 'a', 'good', 'thing', '?', ')', '[SEP]', 'way']	:	['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', None, '_']
['(', 'And', ',', 'by', 'the', 'way', ',', 'is', 'anybody', 'else', 'just', 'a', 'little', 'nostalgic', 'for', 'the', 'days', 'when', 'that', 'was', 'a', 'good', 'thing', '?', ')', '[SEP]', 

Head of test data after process

In [4]:
test_data.head()

,input_form,argument
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, _]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."


## Importing the model and tokenizer

In [25]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Checking the sentence representation

In [45]:
example = test_data['input_form'][1]
print(example)

['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand']


The sentence contains the [SEP] special token followed by the predicate. Therefore, the parameter `add_special_tokens` is set to True so that the index is converted to 102 accordingly and is not treated as another word. \
In addition, the sentence is already split into tokens, to the parameter `is_split_into_words` is also set to True

In [46]:
tokenizer(example,add_special_tokens=True,is_split_into_words=True)

{'input_ids': [101, 2054, 2065, 8224, 4423, 2006, 2049, 3945, 1011, 3194, 1006, 1998, 2085, 1041, 1011, 5653, 1007, 16283, 2015, 2046, 1037, 2440, 1011, 26712, 4082, 2291, 1029, 102, 7818, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [47]:
tokenized_input = tokenizer(example,add_special_tokens=True,is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'what', 'if', 'google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e', '-', 'mail', ')', 'ware', '##s', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand', '[SEP]']


## Tokenizing input and preparing the labels for the model

Converting the labels in the df to numerical values for the language model with `map_labels_in_dataframe` function. \
Add a new column to the df matching the arguments to label numbers. 0 stands for '_' (no argument) and the rest of the arguments are alphabetically ordered. \
*None* label will be mapped to the *[SEP]* token.


In [38]:
train_data = map_labels_in_dataframe(train_data)
dev_data = map_labels_in_dataframe(dev_data)
test_data = map_labels_in_dataframe(test_data)

Checking the head to confirm the labels were correctly converted:

In [39]:
test_data.head()

,input_form,argument,mapped_labels
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, _]","[0, 0, 2, 0, 0, 4, 0, None, 0]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,...","[0, 0, 0, 0, 0, 15, 0, 0, 2, 0, 0, 0, 0, 4, 0,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Use `tokenize_and_align_labels` function to tokenize train, test, and dev dataframe.

In [35]:
tokenized_test = tokenize_and_align_labels(tokenizer, test_data, label_all_tokens=True)
tokenized_train = tokenize_and_align_labels(tokenizer, train_data, label_all_tokens=True)
tokenized_dev = tokenize_and_align_labels(tokenizer, dev_data, label_all_tokens=True)

The input for the model has the corresponding special token [CLS] followed by the tokenized sentence, the special token [SEP], the predicate and the final [SEP] token./
The numerical labels to be fed to the model correspond to the tokenized sentence.

In [48]:
print(tokenizer.convert_ids_to_tokens(tokenized_test["input_ids"][0]))
print(tokenized_test["attention_mask"][0])
print(tokenized_test["input_ids"][0])
print(tokenized_test["labels"][0])

['[CLS]', 'what', 'if', 'google', 'mor', '##ph', '##ed', 'into', 'google', '##os', '?', '[SEP]', 'mor', '##ph', '[SEP]']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[101, 2054, 2065, 8224, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102, 22822, 8458, 102]
[-100, 0, 0, 2, 0, 0, 0, 0, 4, 4, 0, -100, 0, 0, -100]
